In [1]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import random
import numpy as np
import cv2  as cv

In [2]:
!git clone https://github.com/murtazajafferji/svm-gpu.git

Cloning into 'svm-gpu'...
remote: Enumerating objects: 38, done.
remote: Total 38 (delta 0), reused 0 (delta 0), pack-reused 38
Unpacking objects: 100% (38/38), 145.62 KiB | 1.55 MiB/s, done.


In [3]:
train_dir = "../input/autism-image-data/AutismDataset/train"
test_dir = "../input/autism-image-data/AutismDataset/test"

In [4]:
# Getting 'Autistic' and 'Non-Autistic' train images from respective file names of train data
train_non_autistic = []
train_autistic = []
for i in os.listdir(train_dir):
    if 'Non_Autistic' in ("../input/autism-image-data/AutismDataset/train/{}".format(i)):
        train_non_autistic.append(("../input/autism-image-data/AutismDataset/train/{}".format(i)))
    else:
        train_autistic.append(("../input/autism-image-data/AutismDataset/train/{}".format(i)))
        
# Getting test images from test data file path
test_imgs = ["../input/autism-image-data/AutismDataset/test/{}".format(i) for i in os.listdir(test_dir)]


# Concatenate 'Autistic'  and 'Non-Autistic' images and shuffle them as train_images
train_imgs = train_autistic + train_non_autistic
random.shuffle(train_imgs)


In [5]:
# Repeat the above process for validation data to get val_images
val_autistic = "../input/autism-image-data/AutismDataset/valid/Autistic"
val_non_autistic = "../input/autism-image-data/AutismDataset/valid/Non_Autistic"
val_autistic_imgs = ["../input/autism-image-data/AutismDataset/valid/Autistic/{}".format(i) for i in os.listdir(val_autistic)]
val_non_autistic_imgs = ["../input/autism-image-data/AutismDataset/valid/Non_Autistic/{}".format(i) for i in os.listdir(val_non_autistic)]
val_imgs = val_autistic_imgs + val_non_autistic_imgs
random.shuffle(val_imgs)

# Remove the lists to save space
del val_autistic_imgs
del val_non_autistic_imgs

* We get the 'Autistic' and 'Non-Autistic' images from train data, shuffle them and use as training images
* The dataset has 2540 train images, with 1270 images for each 'Autistic' and 'Non-Autistic' category, our dataset is well balanced among the 2 classes
* We also get the 300 test images from file path
* Diffrent images size 
* jpg images

In [6]:
# Set the dimensions for images
nrows = 150
ncolumns  = 150

# Read and process the images: Function returns X,y. X - list of resized images, y - list of labels for the images

def read_and_process_image(list_of_images):
    x = []
    y = []
    
    for image in list_of_images:
        name=image
        image=cv.imread(image, cv.IMREAD_COLOR)
        #image=data_preprocessing(image)
        gray_image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        x.append(cv.resize(gray_image, (nrows, ncolumns)))
        if 'Non_Autistic' in name:
            y.append(0)
        else:
            y.append(1)
    #print(y)   
    #print(x.shape)
    return x,y

In [7]:
x_train, y_train = read_and_process_image(train_imgs)


In [8]:
x_test, y_test = read_and_process_image(test_imgs)

In [9]:
x_val,y_val=read_and_process_image(val_imgs)

In [10]:
x_train = np.array(x_train)

#x_train
nsamples,nx,ny=x_train.shape

In [11]:
x_trainD2=x_train.reshape((nsamples,nx*ny))
x_trainD2.shape

(2540, 22500)

In [12]:
x_test = np.array(x_test)
#x_train
nsamples,nx,ny=x_test.shape

In [13]:
x_testD2=x_test.reshape((nsamples,nx*ny))
x_testD2.shape

(300, 22500)

## Feature Extraction

In [14]:
from sklearn.decomposition import PCA

#initialize PCA with first 10 principal components
pca = PCA(250)
x_train_transformed = pca.fit_transform(x_trainD2)
x_test_transformed=pca.fit_transform(x_testD2)

In [15]:
x_train_transformed.shape

(2540, 250)

In [16]:
x_train_transformed[0]

array([ 3.95701534e+03,  2.67813134e+03, -1.24684454e+03, -2.88197394e+03,
        7.37176765e+01,  3.86686304e+02, -3.37896632e+02, -1.88937253e+03,
       -3.15605108e+02,  1.18672753e+02,  1.50942673e+02, -1.07715811e+03,
       -8.95885640e+02,  7.57055756e+01, -2.07351869e+02, -1.51484526e+03,
       -9.60828460e+02, -9.80950684e+02, -6.25876304e+02, -5.49746232e+02,
        7.72759813e+02,  1.03554792e+03, -2.79257507e+02, -8.34585933e+01,
       -1.11317627e+03,  1.30631105e+03,  8.06331064e+02, -1.03642966e+03,
        3.03881304e+02,  4.20432843e+02,  4.85307067e+02, -7.16793983e+02,
        1.55695161e+02, -2.99898956e+02, -5.67742863e+00, -8.71326844e+02,
        6.08758809e+02, -6.33923163e+02, -3.41964536e+02,  3.12784478e+02,
        1.40437014e+02, -3.13887784e+02,  2.06654355e+02,  9.01420572e+01,
       -1.32871078e+01,  2.82553492e+02,  4.01334031e+02,  6.52566732e+01,
        2.76256180e+02,  8.67109582e+02, -1.88597479e+02, -2.33402237e+02,
       -2.47217252e+02,  

## Data quality

In [17]:
!pip install sewar

  Preparing metadata (setup.py) ... - done
  Created wheel for sewar: filename=sewar-0.4.5-py3-none-any.whl size=10608 sha256=9f5f175373856557cc8d1a170f1063c4bf5cf95a4a7edbfee6e521a51f3353d0
  Stored in directory: /root/.cache/pip/wheels/7a/38/6b/d066cfcb2b1c1c9b059b9ce1bdc4803b469359214e8dad4e3e
Successfully built sewar


In [18]:
 from sewar.full_ref import *

## uqi metric

In [19]:
    index=0
    for image in train_imgs:
        if(index==10):
             break
        y= 0
        test=False
        name=image
        original=image
        type=""
        original=cv.imread(original, cv.IMREAD_COLOR)
        transformedImage=original = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
        img = cv.GaussianBlur(original,(3,3),3)
        laplacian = cv.Laplacian(img,cv.CV_64F)
        sobelx = cv.Sobel(img,cv.CV_64F,1,0,ksize=5)
        if 'Non_Autistic' in name:
            y=0
            type="Non_Autistic"
        else:
            y=1
            type="Autistic"
        print(f"L'image de classe : {type}")
        print(f"le metrique uqi niveau de gray: {uqi(original,transformedImage)}")
        print(f"le metrique uqi niveau de laplacien: {uqi(original,laplacian)}")
        print(f"le metrique uqi niveau de sobelx: {uqi(original,sobelx)}")

        index=index+1
        
        
    

L'image de classe : Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0012364513084843318
le metrique uqi niveau de sobelx: 0.17806395291508478
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0001476399814618108
le metrique uqi niveau de sobelx: 0.27625789588044036
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.00012173912975487145
le metrique uqi niveau de sobelx: 0.32665023036904334
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0002528094777047013
le metrique uqi niveau de sobelx: 0.36494326234423563
L'image de classe : Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0007356506970073577
le metrique uqi niveau de sobelx: 0.29016903156808144
L'image de classe : Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplac

/opt/conda/lib/python3.7/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes uint8 and float64
  warnings.warn(msg)


le metrique uqi niveau de laplacien: 0.0009258469132522364
le metrique uqi niveau de sobelx: 0.2912682635062908
L'image de classe : Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0032067482596592123
le metrique uqi niveau de sobelx: 0.19786897629268418
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.00036579447426481537
le metrique uqi niveau de sobelx: 0.3212439783642211


In [20]:
    index=0
    for image in train_imgs:
        if(index==10):
             break
        y= 0
        test=False
        name=image
        original=image
        type=""
        original=cv.imread(original, cv.IMREAD_COLOR)
        transformedImage=original = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
        img = cv.GaussianBlur(original,(3,3),3)
        laplacian = cv.Laplacian(img,cv.CV_64F)
        sobelx = cv.Sobel(img,cv.CV_64F,1,0,ksize=5)
        if 'Non_Autistic' in name:
            y=0
            type="Non_Autistic"
        else:
            y=1
            type="Autistic"
        print(f"L'image de classe : {type}")
        print(f"le metrique uqi niveau de gray: {uqi(original,transformedImage)}")
        print(f"le metrique uqi niveau de laplacien: {uqi(original,laplacian)}")
        print(f"le metrique uqi niveau de sobelx: {uqi(original,sobelx)}")

        index=index+1
        
        
    

L'image de classe : Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0012364513084843318
le metrique uqi niveau de sobelx: 0.17806395291508478
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0001476399814618108
le metrique uqi niveau de sobelx: 0.27625789588044036
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.00012173912975487145
le metrique uqi niveau de sobelx: 0.32665023036904334
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0002528094777047013
le metrique uqi niveau de sobelx: 0.36494326234423563
L'image de classe : Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplacien: 0.0007356506970073577
le metrique uqi niveau de sobelx: 0.29016903156808144
L'image de classe : Autistic
le metrique uqi niveau de gray: 1.0
le metrique uqi niveau de laplac

## mse

In [21]:
    index=0
    for image in train_imgs:
        if(index==10):
             break
        y= 0
        test=False
        name=image
        original=image
        type=""
        original=cv.imread(original, cv.IMREAD_COLOR)
        transformedImage=original = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
        img = cv.GaussianBlur(original,(3,3),3)
        laplacian = cv.Laplacian(img,cv.CV_64F)
        sobelx = cv.Sobel(img,cv.CV_64F,1,0,ksize=5)
        if 'Non_Autistic' in name:
            y=0
            type="Non_Autistic"
        else:
            y=1
            type="Autistic"
        print(f"L'image de classe : {type}")
        print(f"le metrique uqi niveau de gray: {mse(original,transformedImage)}")
        print(f"le metrique uqi niveau de laplacien: {mse(original,laplacian)}")
        print(f"le metrique uqi niveau de sobelx: {mse(original,sobelx)}")

        index=index+1
        
        
    

L'image de classe : Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 18348.313975346686
le metrique uqi niveau de sobelx: 944639.0205701079
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 20224.791376870802
le metrique uqi niveau de sobelx: 143127.82030331413
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 28737.11578200468
le metrique uqi niveau de sobelx: 317995.8385699348
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 39562.67974361532
le metrique uqi niveau de sobelx: 229218.57686552274
L'image de classe : Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 26449.99854053545
le metrique uqi niveau de sobelx: 433935.65979305946
L'image de classe : Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 27546.823725490198
le

## rmse

In [22]:
    index=0
    for image in train_imgs:
        if(index==10):
             break
        y= 0
        test=False
        name=image
        original=image
        type=""
        original=cv.imread(original, cv.IMREAD_COLOR)
        transformedImage=original = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
        img = cv.GaussianBlur(original,(3,3),3)
        laplacian = cv.Laplacian(img,cv.CV_64F)
        sobelx = cv.Sobel(img,cv.CV_64F,1,0,ksize=5)
        if 'Non_Autistic' in name:
            y=0
            type="Non_Autistic"
        else:
            y=1
            type="Autistic"
        print(f"L'image de classe : {type}")
        print(f"le metrique uqi niveau de gray: {rmse(original,transformedImage)}")
        print(f"le metrique uqi niveau de laplacien: {rmse(original,laplacian)}")
        print(f"le metrique uqi niveau de sobelx: {rmse(original,sobelx)}")

        index=index+1
        
        
    

L'image de classe : Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 135.45594846793065
le metrique uqi niveau de sobelx: 971.925419242705
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 142.21389305152573
le metrique uqi niveau de sobelx: 378.32237615995456
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 169.52025183441853
le metrique uqi niveau de sobelx: 563.9111974149253
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 198.90369464546234
le metrique uqi niveau de sobelx: 478.7677692425867
L'image de classe : Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 162.63455518596118
le metrique uqi niveau de sobelx: 658.7379295236152
L'image de classe : Autistic
le metrique uqi niveau de gray: 0.0
le metrique uqi niveau de laplacien: 165.972358317553
le m

## psnr

In [23]:
    index=0
    for image in train_imgs:
        if(index==10):
             break
        y= 0
        test=False
        name=image
        original=image
        type=""
        original=cv.imread(original, cv.IMREAD_COLOR)
        transformedImage=original = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
        img = cv.GaussianBlur(original,(3,3),3)
        laplacian = cv.Laplacian(img,cv.CV_64F)
        sobelx = cv.Sobel(img,cv.CV_64F,1,0,ksize=5)
        if 'Non_Autistic' in name:
            y=0
            type="Non_Autistic"
        else:
            y=1
            type="Autistic"
        print(f"L'image de classe : {type}")
        print(f"le metrique uqi niveau de gray: {psnr(original,transformedImage)}")
        print(f"le metrique uqi niveau de laplacien: {psnr(original,laplacian)}")
        print(f"le metrique uqi niveau de sobelx: {psnr(original,sobelx)}")

        index=index+1
        
        
    

L'image de classe : Autistic
le metrique uqi niveau de gray: inf
le metrique uqi niveau de laplacien: 5.494841977209237
le metrique uqi niveau de sobelx: -11.621855203137699
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: inf
le metrique uqi niveau de laplacien: 5.07196310402691
le metrique uqi niveau de sobelx: -3.426436965809361
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: inf
le metrique uqi niveau de laplacien: 3.5463718311065158
le metrique uqi niveau de sobelx: -6.893410757896684
L'image de classe : Non_Autistic
le metrique uqi niveau de gray: inf
le metrique uqi niveau de laplacien: 2.1579466038842723
le metrique uqi niveau de sobelx: -5.471694509582134
L'image de classe : Autistic
le metrique uqi niveau de gray: inf
le metrique uqi niveau de laplacien: 3.9065470846031323
le metrique uqi niveau de sobelx: -8.243449800080127
L'image de classe : Autistic
le metrique uqi niveau de gray: inf
le metrique uqi niveau de laplacien: 3.7300883092937465

## Modeling 


### Xgboost with pca

In [24]:
from xgboost import XGBClassifier
xboost = XGBClassifier(n_estimators = 700, learning_rate = 0.1,gpu_id=1)
xboost.fit(x_train_transformed,y_train)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=700,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [25]:
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred_X =xboost.predict(x_test_transformed)
print(y_pred_X)
cm_X = confusion_matrix(y_test,y_pred_X)
print('Accuracy == {0}'.format(accuracy_score(y_test,y_pred_X)))
print(cm_X)

[1 0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 0 0
 1 1 1 0 1 1 1 0 0 1 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 1 1 0 1 1
 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 1
 1 0 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0
 1 1 1 1 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0
 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 0 0 0 1 1 0 1 1
 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 1 1
 1 0 0 0]
Accuracy == 0.6133333333333333
[[85 65]
 [51 99]]


### KNN with pca

In [26]:

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
model.fit(x_train_transformed,y_train)



KNeighborsClassifier()

In [27]:

#Predict Output
y_pred_KNN= model.predict(x_test_transformed) 


In [28]:
cm_X = confusion_matrix(y_test,y_pred_KNN)
print('Accuracy == {0}'.format(accuracy_score(y_test,y_pred_KNN)))
print(cm_X)

Accuracy == 0.5933333333333334
[[129  21]
 [101  49]]
